# Comparação entre dados de COVID-19 divulgados para a cidade de Indaiatuba entre Governo SP e Prefeitura. 

Neste projeto iremos fazer uma comparação entre os dados do numero de casos e obitos divulgados entre a prefeitura de Indaiatuba e os dados divulgados pelo governo do estado de SP. No Brasil o numero de casos do novo coronavirus vem crescendo dia após dia, sendo hoje o segundo país do mundo com o maior numero casos de COVID-19 (25/05/2020), e o estado de São Paulo é o estado com o maior numero de casos e obitos quando comparamos os valores absolutos.

Quando analisamos dados municipais em relação aos dados do estado de SP é possivel ver uma disparidade entre os valores, uma reportagem da Globo foi feita questionando esses dados para municipios da Grade SP - https://glo.bo/3bZbHjC, em nota a secretaria disse que os municípios são responsáveis pela investigação e notificação dos casos de Covid-19, e que os horarios de notificação do estado e dos municipios são diferentes, por isso pode difererir.

Como a prefeitura não disponibiliza os dados em formato de arquivo para download, então é necessario fazer um web scrapy das noticias postadas no site da prefeitura https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/acoes-e-obras/acao/1216/ e para os dados do governo de São Paulo está disponivel no repósitorio do github https://github.com/seade-R/dados-covid-sp.

# Pacotes e Retirando as descrições dos Sites.

In [1]:
#Bibliotecas para fazer web scrapy dos sites
import requests
from requests import get
from bs4 import BeautifulSoup

#trabalhar com data frame
import pandas as pd

#Biblioteca para o processamento de linguagem
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import bigrams

import datetime
import numpy as np

stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gusta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Para a primeira parte do processo, foi utilizado a url contendo todas as notícias relacionda a cidade de Indaiatuba, com isso separamos as notícias relacionado a COVID-19 que contêm dados com óbitos e casos confirmados.

In [2]:
url = 'https://www.indaiatuba.sp.gov.br/saude/vigilancia-em-saude/vigilancia-epidemiologica/novo-coronavirus/notas/'

results = requests.get(url)

soup = BeautifulSoup(results.text, "html.parser")

sites = []

div_sit = soup.find_all('div', class_='grid-75 content')

div_sit2 = div_sit[0].find_all('ul')[1]

for sit in div_sit2:
    hr = sit.find('a',href = True)
    sites.append('https://www.indaiatuba.sp.gov.br' + hr['href'])

In [4]:
#Removendo sites que não tem dados interessantes.
sites.remove("https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28430/")
sites.remove("https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28486/")
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28327/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28329/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28333/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28335/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28336/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28339/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28341/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28342/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28350/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28352/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28354/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28355/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28357/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28361/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28365/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28372/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28374/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28386/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28389/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28426/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28516/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28676/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28712/')
sites.remove('https://www.indaiatuba.sp.gov.br/relacoes-institucionais/imprensa/noticias/28920/')

In [7]:
descricao = []
datas = []

for i in sites:
  url1 = i
  results1 = requests.get(url1)

  soup1 = BeautifulSoup(results1.text, "html.parser")

  div_sit1 = soup1.find_all('div', class_='grid-75 content')

  for container1 in div_sit1:
    
    inside_ul = container1.find_all('ul', {'class':'info_noticia'})
    get_all_text = inside_ul[0].find('li').text
    finda_date = re.search(r'\d{2}/\d{2}/\d{4}',get_all_text)
    datas.append(finda_date.group())

    str_free = ''
    aux_des = container1.findAll('p')

    for strin in aux_des:
      str_free = str_free + ' ' + strin.text
 
    descricao.append(str_free)

# Dados pessoais

In [182]:
#Encontrando as comorbidades dos obtiso
Morb = []
diab = []
hiperten = []
pulm = []
card = []
renal = []
obes = []

tv = []
for k in range(0,len(posi)):
    aux_age = age[k]
    spli = descricao[posi[k]].replace(',','').replace('.',' .').replace('"','').replace('-'," ").replace(')','').lower().split()
    flag = 0

    for i in range(0,len(spli)-1):
        if flag == 0:
            if (spli[i] == aux_age) & (spli[i+1] == 'anos'):
    
                tes = spli[i-10:i+20]
                tv.append(posi[k])
                flag += 1
                if 'crônicas' in tes:
                    pos_co = tes.index('crônicas')
                    new_te = tes[pos_co - 5:pos_co]
                    if ('não' in new_te) | ('sem' in new_te):
                        Morb.append(0)
                    else:
                        Morb.append(1)
                elif 'comorbidade' in tes:
                    pos_co = tes.index('comorbidade')
                    new_te = tes[pos_co - 5:pos_co]
                    if ('não' in new_te) | ('sem' in new_te):
                        Morb.append(0)
                    else:
                        Morb.append(1)

                elif 'comorbidades' in tes:
                    pos_co = tes.index('comorbidades')
                    new_te = tes[pos_co - 5:pos_co]
                    if ('não' in new_te) | ('sem' in new_te):
                        Morb.append(0)
                    else:
                        Morb.append(1)
                else:
                     Morb.append(1)

                if 'diabetes' in tes:
                    diab.append(1)
                else:
                    diab.append(0)

                if ('hipertensão' in tes) | ('hipertenso' in tes) | ('hipertensa' in tes):
                    hiperten.append(1)
                else:
                    hiperten.append(0)

                if ('cardíaco' in tes) | ('cardíaca' in tes):
                    card.append(1)
                else:
                    card.append(0)

                if ('pulmonar' in tes) | ('pulmão' in tes) | ('tabagista' in tes):
                    pulm.append(1)
                else:
                    pulm.append(0)

                if ('renal' in tes):
                    renal.append(1)
                else:
                    renal.append(0)

                if('obesidade' in tes) | ('obesa' in tes) | ('obeso' in tes):
                    obes.append(1)
                else:
                    obes.append(0)
#'hipertensão' in tes

In [12]:
#Encontrando o genero, idade e data do acontecido
num2words = {'um':1,'dois' : 2,'três': 3, 'quatro':4,'cinco':5, \
             'seis':6, 'sete':7,'oito' : 8,'nove':9,'dez':10,'zero': 0}

age = []
count = []
gender = []
Dat = []
posi = []
for j in range(0,209):   
    spli1 = descricao[j].replace(',','').replace('.','').replace('"','').lower().split()
    
    spli = [word for word in spli1 if not word in stopwords]
    cou = 0
    for i in range(0,len(spli)):
        if spli[i] == 'anos':
            if (spli[i-2] == 'feminina') | (spli[i-2] == 'mulher') | (spli[i-2] == 'feminino'):
                    cou = cou + 1
                    #print("iDADE: {}".format(spli[i-1]))
                    #print(j)
                    age.append(spli[i-1])
                    gender.append('M')
                    Dat.append(datas[j])
                    posi.append(j)
                    #count.append(k)
            if (spli[i-2] == 'masculino') | (spli[i-2] == 'homem') | (spli[i-2] == 'agosto') | (spli[i-2]=='silva'):
                    cou = cou + 1
                    #print("iDADE: {}".format(spli[i-1]))
                    #print(j)
                    age.append(spli[i-1])
                    gender.append('H')
                    Dat.append(datas[j])
                    posi.append(j)
            if  (spli[i-2] == 'paciente'):
                    cou = cou + 1
                    #print("iDADE: {}".format(spli[i-1]))
                    #print(j)
                    age.append(spli[i-1])
                    Dat.append(datas[j])
                    posi.append(j)
                    if spli[i+1] == 'internado':
                        gender.append('H')
                    else:
                        gender.append('M')
                    
    count.append(cou)
                    
#ind_beg = ind.index(193) +2
#ind_end = ind_beg + 6
#index_dele = list(range(ind_beg, ind_end))
#del age[236:ind_end]
#for k in range(0,len(index_dele)):
#    age.pop(index_dele[k])
del age[238:244]
del gender[238:244]
del Dat[238:244]
del posi[238:244]
count[193] = count[193] -6

In [184]:
dict_pero = {'Idade':age,'Sexo':gender,'Data':Dat,'Comorbidade':Morb,'Diabete':diab,'Hipertensão':hiperten,
            'Pulmonar':pulm,'Cardiaco':card,'Renal':renal,'Obesidade':obes}

In [185]:
pd.DataFrame(dict_pero).head(20)

,Idade,Sexo,Data,Comorbidade,Diabete,Hipertensão,Pulmonar,Cardiaco,Renal,Obesidade
0,60,H,10/12/2020,1,1,1,0,0,0,0
1,73,H,09/12/2020,1,1,1,0,0,0,0
2,65,H,09/12/2020,1,0,1,0,0,0,0
3,66,H,08/12/2020,1,0,1,0,0,1,0
4,64,H,07/12/2020,0,0,0,0,0,0,0
5,65,H,07/12/2020,0,0,0,1,0,0,0
6,55,M,07/12/2020,1,0,0,0,0,0,0
7,81,H,07/12/2020,1,1,1,0,0,0,0
8,46,H,05/12/2020,1,0,0,0,0,1,0
9,85,M,03/12/2020,1,0,0,0,0,0,0


In [186]:
pd.DataFrame(dict_pero).to_excel('Perso_Covid.xlsx')

In [187]:
pd.DataFrame(dict_pero).to_csv('Perso_Covid.csv', index = False)

# Dados Temporal

In [162]:
UTI_vect = []

stopwords = nltk.corpus.stopwords.words('portuguese')
for i in range(0,len(descricao) - 10):
    spli1 = descricao[i].replace(',','').replace('.','').replace("'",'').lower().split()
    spli = [word for word in spli1 if not word in stopwords]

    bi = list(bigrams(spli))
    values = []
    for j in range(0,len(bi)):
        if (bi[j][0] == 'uti') & (bi[j][1] == '(unidade'):
            values.append(spli[j-1])
    UTI_vect.append(values)
    
UTI_vect[190] = UTI_vect[189]
UTI_vect[197] = [str(12)]
UTI_vect[198:len(UTI_vect)] = np.repeat([str(0)],len(UTI_vect)-198)

In [163]:
UTI = []

for i in range(0,len(UTI_vect)):
    next_aux = np.unique(UTI_vect[i])

    UTI.append(int(next_aux[0]))
  

In [164]:
num2words = {'um':1,'dois' : 2,'três': 3, 'quatro':4,'cinco':5, \
             'seis':6, 'sete':7,'oito' : 8,'nove':9,'dez':10,'zero': 0}

leitos = []

stopwords = nltk.corpus.stopwords.words('portuguese')
for i in range(0,len(descricao) - 10):
    spli1 = descricao[i].replace(',','').replace('.','').replace('confirmados','').lower().split()
    spli = [word for word in spli1 if not word in stopwords]

    bi = list(bigrams(spli))
    values = []
    for j in range(0,len(bi)):
        if (bi[j][0] == 'leitos') & (bi[j][1] == 'clínicos'):
            values.append(spli[j-1])
        if (bi[j][0] == 'permanecem') & (bi[j][1] == 'internados'):
            if spli[j-1] in num2words.keys():
                values.append(num2words[spli[j-1]])
            else:
                values.append(spli[j-1])
        if (bi[j][0] == 'leito') & (bi[j][1] == 'clínico'):
            if spli[j-1] in num2words.keys():
                values.append(num2words[spli[j-1]])
            else:
                values.append(spli[j-1])
        if (bi[j][0] == 'internados') & (bi[j][1] == 'ainda'):
            if spli[j-1] in num2words.keys():
                values.append(num2words[spli[j-1]])
            else:
                values.append(spli[j-1])
        if (bi[j][0] == 'internados') & (bi[j][1] == 'leito'):
            values.append(spli[j-1])
        if (bi[j][0] == 'pessoas') & (bi[j][1] == 'internadas'):
            values.append(spli[j-1])
        if (bi[j][0] == 'casos') & (bi[j][1] == 'internados'):
            values.append(spli[j-1]) 
    leitos.append(values)

In [165]:
internados = []
leitos[3][0] = leitos[3][0].replace('estão','')
leitos[91].append(33)
for i in range(0,len(leitos)):
    aux = np.unique(leitos[i])
    
    if len(aux) == 1:
        next_aux = aux
    else:    
        next_aux = [int(new.replace('.','')) for new in aux if new.replace('.','').isnumeric()]

    if len(next_aux) == 1:
        internados.append(next_aux[0])
        #confirmados.append(int(np.unique(next_aux[i])[0].replace('.','')))
    else:
        internados.append(max(next_aux))

In [166]:
conf_val = []
stopwords = nltk.corpus.stopwords.words('portuguese')
for i in range(0,len(descricao) - 10):
    
    
    spli1 = descricao[i].replace(',','').lower().split()
    
    spli = [word for word in spli1 if not word in stopwords]
    
    bi = list(bigrams(spli))
    values = []
    for j in range(0,len(bi)):
        if (bi[j][0] == 'casos') & (bi[j][1] == 'confirmados'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'caso') & (bi[j][1] == 'confirmado'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'pessoas') & (bi[j][1] == 'infectadas'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'pessoas') & (bi[j][1] == 'contraíram'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'indaiatuba') & (bi[j][1] == 'possui'):
            #print(i)
            values.append(spli[j+2])
        elif (bi[j][0] == 'infectados') & (bi[j][1] == 'novo'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'confirmados') & (bi[j][1] == 'covid-19'):
            #print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'confirmados') & (bi[j][1] == 'sendo'):
           # print(i)
            values.append(spli[j-1])
        elif (bi[j][0] == 'notificações;') & (bi[j][1] == 'sendo'):
            #print(i)
            values.append(spli[j+2]) 
    conf_val.append(values)

In [167]:
confirmados = []
for i in range(0,len(conf_val)):
    #print(i)
    aux = np.unique(conf_val[i])

    next_aux = [int(new.replace('.','')) for new in aux if new.replace('.','').isnumeric()]

    if len(next_aux) == 1:
        confirmados.append(next_aux[0])
        #confirmados.append(int(np.unique(next_aux[i])[0].replace('.','')))
    else:
        next_aux3 = [int(aux3)/confirmados[i-1] for aux3 in next_aux]
        max_pos = next_aux3.index(max(next_aux3))
        confirmados.append(next_aux[max_pos])

In [168]:
obi_val = []
stopwords = nltk.corpus.stopwords.words('portuguese')
for i in range(0,len(descricao) - 10):
    
    
    spli1 = descricao[i].replace(',','').replace(';','').replace('.','').lower().split()
    
    spli = [word for word in spli1 if not word in stopwords]
    
    bi = list(bigrams(spli))
    values = []
    for j in range(0,len(bi)):
        if (bi[j][0] == 'óbitos') & (bi[j][1] == 'confirmados'):
            values.append(spli[j-1])
            #print(spli[j-1])
        if (bi[j][0] == 'veio') & (bi[j][1] == 'óbito'):
            values.append(spli[j-1])
        if (bi[j][0] == 'vieram') & (bi[j][1] == 'óbito'):
            values.append(spli[j-1])
        if (bi[j][0] == 'evoluíram') & (bi[j][1] == 'óbito'):
            values.append(spli[j-1])
        if (bi[j][0] == 'resistiram') & (bi[j][1] == 'complicações'):
            values.append(spli[j-1])
        if (bi[j][0] == 'total') & (bi[j][1] == 'óbitos'):
            values.append(spli[j-1])
        if (bi[j][0] == 'total') & (bi[j][1] == 'departamento'):
            values.append(spli[j-1])     
            #print(spli[j-1])
    obi_val.append(values)


num2words = {'um':1,'dois' : 2,'três': 3, 'quatro':4,'cinco':5, \
             'seis':6, 'sete':7,'oito' : 8,'nove':9,'dez':10,'zero': 0}

for j in range(200,204):
    if len(obi_val[j]) == 1:
        #
        obi_val[j] = [str(num2words[obi_val[j][0]])]
        #print(num2words[obi_val[j][0]])
    else:
        obi_val[j] = [str(num2words[obi_val[j][1]])]
        #print(num2words[obi_val[j][1]])

In [169]:
obitos = []
for i in range(0,len(obi_val)):
    #print(i)
    aux = np.unique(obi_val[i])

    next_aux = [int(new.replace('.','')) for new in aux if new.replace('.','').isnumeric()]

    if len(next_aux) == 1:
        obitos.append(next_aux[0])
        #confirmados.append(int(np.unique(next_aux[i])[0].replace('.','')))
    else:
        next_aux3 = [int(aux3)/obitos[i-1] for aux3 in next_aux]
        max_pos = next_aux3.index(max(next_aux3))
        obitos.append(next_aux[max_pos])

In [170]:
confirm_beg = confirmados[::-1]
obito_beg = obitos[::-1]
datas_beg = datas[:204][::-1]
internado_beg = internados[::-1]
Uti_beg = UTI[::-1]

confirmado_per_day = []
obito_per_day = []

for k in range(0,len(confirm_beg)-1):

    confirmado_per_day.append(confirm_beg[k+1] - confirm_beg[k])
    obito_per_day.append(abs(obito_beg[k+1] - obito_beg[k]))

confirmado_per_day.insert(0,confirm_beg[0])
obito_per_day.insert(0,obito_beg[0])
#datas_beg = datas_beg[:204]

In [171]:
dict_pd = {'Data':datas_beg,'Confirmados_total':confirm_beg,'Confirmados_por_dia':confirmado_per_day,
          'Obitos_total':obito_beg,'Obitos_por_dia':obito_per_day,'Internados':internado_beg,'UTI':Uti_beg}

Df_prefeitura = pd.DataFrame(dict_pd)

In [172]:
Df_prefeitura['Letalidade_Indaiatuba'] = (Df_prefeitura['Obitos_total'].values/Df_prefeitura['Confirmados_total'].values)*100
Df_prefeitura['Letalidade_Indaiatuba']  = Df_prefeitura['Letalidade_Indaiatuba'].apply(lambda x: round(x,2))

In [173]:
Df_prefeitura

,Data,Confirmados_total,Confirmados_por_dia,Obitos_total,Obitos_por_dia,Internados,UTI,Letalidade_Indaiatuba
0,02/05/2020,58,58,6,6,3,0,10.34
1,06/05/2020,69,11,7,1,5,0,10.14
2,07/05/2020,71,2,8,1,4,0,11.27
3,08/05/2020,78,7,9,1,7,0,11.54
4,11/05/2020,87,9,11,2,11,0,12.64
...,...,...,...,...,...,...,...,...
199,07/12/2020,9765,47,255,1,48,30,2.61
200,07/12/2020,9827,62,258,3,46,28,2.63
201,08/12/2020,9971,144,259,1,40,27,2.60
202,09/12/2020,10036,65,261,2,43,26,2.60


# Dados Governo

Nessa primeira abordagem utilizando web scrape foi retirado apenas informações relacionadas ao número de Casos e de óbitos disponibilizados pelo site da prefeitura de Indaiatuba.
Vamos Transformar em um Data Frame, além disso podemos calcular o número de casos e óbitos por dia, apenas fazendo a subtração dos valores no tempo
t - (t-1)

Para fazer a comparação com os dados do governo do estado de SP, vamos utilizar os dados disponiveis no repositorio do github do Seade-SP https://github.com/seade-R/dados-covid-sp.
O conjunto de dados contem valores para todos os municipios do estado de SP, e o formato do tempo que estão utilizando é o dos Estados unidos, logo foi feitos alguns ajustes para trabalhar com os dados.

In [190]:
#Chamando os dados do Governo no github 
Dados_governo = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv',sep = ';',decimal=",")

#Mudando o formato de data que estava 2020-04-24 para o nosso 24/04/2020.
Date_wrong_format = Dados_governo.datahora.values
Date_right = []
for date in Date_wrong_format:
  Date_right.append(datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%d/%m/%Y'))

Dados_governo.datahora = Date_right

Como podemos notar o conjunto de dados do governo tem dados de todos os municipios, vamos separar para Indaiatuba. Deixando apenas as colunas de Data , Casos Positivos, Casos Positivos por dia, Óbito e Óbitos por dia

In [191]:
#Como nossa comparação é com o municipio de indaiatuba, vamos separar nossos dados
Dados_governo2 = Dados_governo.query("nome_munic == 'Indaiatuba'")
Dados_Governo_Indaiatuba = Dados_governo2[["datahora","casos","casos_novos","obitos","obitos_novos","letalidade"]]
Dados_Governo_Indaiatuba.columns = ["Data","Casos_Positivos","Casos_por_dia","Obitos","Obitos_por_dia","Letalidade"]
Dados_Governo_Indaiatuba.Letalidade = Dados_Governo_Indaiatuba.Letalidade*100

Dados_Governo_Indaiatuba_final = Dados_Governo_Indaiatuba[Dados_Governo_Indaiatuba['Data'].isin(Df_prefeitura['Data'].values)]

D:\Users\gusta\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [192]:
Df_final = Df_prefeitura.merge(Dados_Governo_Indaiatuba_final,how = 'left', on = 'Data')

In [195]:
Df_final = Df_final.rename(columns = {'Confirmados_total':'Confirmados_total_Prefeitura',
               'Confirmados_por_dia':'Confirmados_dia_Prefeitura','Obitos_total':'Obitos_total_Prefeitura',
                'Obitos_por_dia_x':'Obitos_dia_Prefeitura','Casos_Positivos':'Confirmados_total_Governo',
                'Casos_por_dia':'Casos_dia_Governo','Obitos':'Obitos_Total_Governo','Obitos_por_dia_y':'Obitos_dia_Governo',
                 'Letalidade':'Letalidade_Governo','Letalidade_Indaiatuba':'Letalidade_Prefeitura'})

In [47]:
Df_final.to_csv('COVID_comp.csv')

In [196]:
Df_final.to_excel('COVID_comp.xlsx')